In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pickle

In [8]:
# load the trained model, scaler pickle and onehot

model=load_model('model.h5')

## load encoders and sclaer

with open('label_categorical_gender.pkl','rb') as file:
    label_categorical_gender=pickle.load(file)

with open('onehot_encode_geo.pkl','rb') as file:
    onehot_encode_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [9]:
## Example input data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [12]:
## one-hot encode 'Geography'
geo_encoded=onehot_encode_geo.fit_transform([[input_data['Geography']]])
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encode_geo.get_feature_names_out(['Geography']))



In [13]:
geo_encoded_df

,Geography_France
0,1.0


In [14]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [15]:
## Encode
input_df['Gender']=label_categorical_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [16]:
## combine with input data
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France
0,600,1,40,3,60000,2,1,1,50000,1.0


In [20]:
input_scaler=scaler.transform(input_df)
input_scaler

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [21]:
prediction=model.predict(input_scaler)
prediction

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 12, but received input with shape (1, 10)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 10), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>

prediction_prob=prediction[0][0]


In [22]:
if prediction_prob>0.5:
    print('likely to churn')
else:
    print('not likely to churn')

NameError: name 'prediction_prob' is not defined